In [ ]:
from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import ast
import datetime
pd.set_option('display.max_columns', None)

### Functions definitions

In [ ]:
# For data exploration
def data_summary(df, cat_cols, num_cols, describe_pts=[0.1, .2,.25,.5, .75, .9, .95, .99]):
    print("Data shape: ", df.shape)
    print("Data columns: ", df.columns)

    # Get hit rates for variables in the dataset, evaluate missing values proportions
    hr = df.notnull().mean().reset_index().rename(columns={'index':'variable',0:'hit_rate'})

    # Get data types
    df_types = df.dtypes.reset_index().rename(columns={'index':'variable',0:'data_type'})

    # Put info in same data add
    variable_summary = pd.merge(df_types, hr, on=['variable'], how='left', suffixes=('','_hr'))

    # Get value counts for categorical variables
    cat_vars_summary = pd.DataFrame(columns=['variable_name','unique_values','counts'])
    for col in cat_cols:
        new_counts = df[col].value_counts(dropna=False).rename_axis('unique_values').to_frame('counts').reset_index()
        new_counts.insert(loc=0, column='variable_name', value=col)
        cat_vars_summary = pd.concat([cat_vars_summary, new_counts], ignore_index = True)

    cont_vars_summary = df[num_cols].describe(describe_pts).reset_index().rename(columns={'index':'stats'})

    # Get correlation matrix for continuous variables
    df_corr = df[num_cols].corr()

    return variable_summary, cat_vars_summary, cont_vars_summary, df_corr

In [ ]:
def summarize(df: pd.DataFrame, level: list, cols: list) -> pd.DataFrame:
    aggregations = {}
    for col in cols:
        aggregations.update({col:  [("Min", "min"), ("Max", "max"), ("Mean", "mean"), ("Sum","sum")]})

    df_agg = df.groupby(level).agg(aggregations).reset_index()

    # Renaming columns
    df_agg.columns = ["_".join(col).strip() for col in df_agg.columns.values]

    return df_agg

### Data Import

In [ ]:
# Data dictionary can be found here: https://www.kaggle.com/datasets/nadyinky/sephora-products-and-skincare-reviews

In [ ]:
# Navigate to your data directory
# data_dir = '/content/drive/MyDrive/Colab Notebooks/DermaLLM/data/'
data_dir = '/content/drive/MyDrive/Colab Notebooks/DermaLLM_old_backup/DermaLLM/data/'

td_date = datetime.datetime.now().strftime("%m_%d_%Y")

In [ ]:
# Import reviews table
reviews_df = pd.read_pickle(data_dir + "reviews_info.pkl")
# Drop extra index column
reviews_df.drop(['Unnamed: 0'], inplace=True, axis=1)
print(reviews_df.shape)
# Drop initial duplicates
reviews_df.drop_duplicates(inplace=True)
print(reviews_df.shape)
print(reviews_df.columns)
reviews_df

(1094411, 18)
(1094187, 18)
Index(['author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name', 'brand_name', 'price_usd'],
      dtype='object')


,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
0,1741593524,5,1.0,1.0,2,0,2,2023-02-01,I use this with the Nudestix “Citrus Clean Bal...,Taught me how to double cleanse!,NaN,brown,dry,black,P504322,Gentle Hydra-Gel Face Cleanser,NUDESTIX,19.0
1,31423088263,1,0.0,NaN,0,0,0,2023-03-21,I bought this lip mask after reading the revie...,Disappointed,NaN,NaN,NaN,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
2,5061282401,5,1.0,NaN,0,0,0,2023-03-21,My review title says it all! I get so excited ...,New Favorite Routine,light,brown,dry,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
3,6083038851,5,1.0,NaN,0,0,0,2023-03-20,I’ve always loved this formula for a long time...,Can't go wrong with any of them,NaN,brown,combination,black,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
4,47056667835,5,1.0,NaN,0,0,0,2023-03-20,"If you have dry cracked lips, this is a must h...",A must have !!!,light,hazel,combination,NaN,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49972,2276253200,5,1.0,NaN,0,0,0,2023-03-13,Consider salicylic acid your secret weapon for...,NaN,fair,brown,combination,NaN,P505392,Multi Action Clear Acne Clearing Treatment Lot...,StriVectin,49.0
49973,28013163278,5,1.0,NaN,0,0,0,2023-03-13,I’ve been using this as my only moisturizer fo...,NaN,NaN,blue,combination,blonde,P505392,Multi Action Clear Acne Clearing Treatment Lot...,StriVectin,49.0
49974,1539813076,5,1.0,NaN,0,0,0,2023-03-13,I got breakouts whenever it’s my time of month...,NaN,light,blue,combination,blonde,P505392,Multi Action Clear Acne Clearing Treatment Lot...,StriVectin,49.0
49975,5595682861,5,1.0,NaN,0,0,0,2023-03-13,I love this!!! I don’t get actual acne just an...,NaN,fair,hazel,oily,NaN,P505392,Multi Action Clear Acne Clearing Treatment Lot...,StriVectin,49.0


In [ ]:
numerical_cols = reviews_df.select_dtypes(include='number').columns.tolist()
categorical_cols = reviews_df.select_dtypes(include='object').columns.tolist()

In [ ]:
variable_summary, cat_vars_summary, cont_vars_summary, df_corr = data_summary(reviews_df, cat_cols = categorical_cols , num_cols = numerical_cols)

Data shape:  (1094187, 18)
Data columns:  Index(['author_id', 'rating', 'is_recommended', 'helpfulness',
       'total_feedback_count', 'total_neg_feedback_count',
       'total_pos_feedback_count', 'submission_time', 'review_text',
       'review_title', 'skin_tone', 'eye_color', 'skin_type', 'hair_color',
       'product_id', 'product_name', 'brand_name', 'price_usd'],
      dtype='object')


In [ ]:
variable_summary

,variable,data_type,hit_rate
0,author_id,object,1.000000
1,rating,int64,1.000000
2,is_recommended,float64,0.846577
3,helpfulness,float64,0.486865
4,total_feedback_count,int64,1.000000
5,total_neg_feedback_count,int64,1.000000
6,total_pos_feedback_count,int64,1.000000
7,submission_time,object,1.000000
8,review_text,object,0.998680
9,review_title,object,0.716113


In [ ]:
cat_vars_summary

,variable_name,unique_values,counts
0,author_id,1696370280,234
1,author_id,1288462295,171
2,author_id,5060164185,132
3,author_id,2330399812,131
4,author_id,7375781965,130
...,...,...,...
1922353,brand_name,TWEEZERMAN,38
1922354,brand_name,Soleil Toujours,37
1922355,brand_name,Anastasia Beverly Hills,27
1922356,brand_name,caliray,24


In [ ]:
cont_vars_summary

,stats,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,price_usd
0,count,1.094187e+06,926314.000000,532721.000000,1.094187e+06,1.094187e+06,1.094187e+06,1.094187e+06
1,mean,4.299077e+00,0.839954,0.767891,4.177699e+00,8.948132e-01,3.282886e+00,4.900874e+01
2,std,1.149490e+00,0.366649,0.317057,2.271751e+01,5.289416e+00,1.967678e+01,4.004439e+01
3,min,1.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
4,10%,2.000000e+00,0.000000,0.200000,0.000000e+00,0.000000e+00,0.000000e+00,1.400000e+01
5,20%,4.000000e+00,1.000000,0.500000,0.000000e+00,0.000000e+00,0.000000e+00,2.200000e+01
6,25%,4.000000e+00,1.000000,0.653846,0.000000e+00,0.000000e+00,0.000000e+00,2.500000e+01
7,50%,5.000000e+00,1.000000,0.928571,0.000000e+00,0.000000e+00,0.000000e+00,3.900000e+01
8,75%,5.000000e+00,1.000000,1.000000,3.000000e+00,1.000000e+00,3.000000e+00,6.200000e+01
9,90%,5.000000e+00,1.000000,1.000000,1.000000e+01,2.000000e+00,8.000000e+00,8.900000e+01


In [ ]:
# Quick check - how many duplicates exist?
print(f"Original rows: {len(reviews_df)}")
print(f"Unique rows (all columns): {len(reviews_df.drop_duplicates())}")
print(f"Duplicate rows: {len(reviews_df) - len(reviews_df.drop_duplicates())}")

# Check individual column uniqueness
print("\nUniqueness per column:")
for col in reviews_df.columns:
    unique_pct = (reviews_df[col].nunique() / len(reviews_df)) * 100
    print(f"{col}: {reviews_df[col].nunique()} unique ({unique_pct:.1f}%)")

Original rows: 1094187
Unique rows (all columns): 1094187
Duplicate rows: 0

Uniqueness per column:
author_id: 578653 unique (52.9%)
rating: 5 unique (0.0%)
is_recommended: 2 unique (0.0%)
helpfulness: 3767 unique (0.3%)
total_feedback_count: 676 unique (0.1%)
total_neg_feedback_count: 259 unique (0.0%)
total_pos_feedback_count: 590 unique (0.1%)
submission_time: 5317 unique (0.5%)
review_text: 969419 unique (88.6%)
review_title: 364105 unique (33.3%)
skin_tone: 14 unique (0.0%)
eye_color: 6 unique (0.0%)
skin_type: 4 unique (0.0%)
hair_color: 7 unique (0.0%)
product_id: 2351 unique (0.2%)
product_name: 2334 unique (0.2%)
brand_name: 142 unique (0.0%)
price_usd: 221 unique (0.0%)


In [ ]:
check = reviews_df[reviews_df['product_id']=='P377368']
check

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
91964,1568079543,5,1.0,1.000000,4,0,4,2022-08-12,I have problem skin and breakout easy. After ...,It changed my life,lightMedium,brown,combination,black,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
91965,5718657317,5,1.0,0.714286,7,2,5,2022-02-14,I was very skeptical about this one but I had ...,miracle worker,light,brown,combination,brown,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
91966,32245620971,2,0.0,0.812500,16,3,13,2021-12-26,"It feels nice on skin, easy to use, but I hone...",Doesn't work,fairLight,hazel,oily,auburn,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
91967,2500599340,2,0.0,0.772727,22,5,17,2021-07-12,"If you have pitted (scooped out) scars, this w...",For Pitted Scars -- This Ain't It.,light,brown,dry,brown,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
91968,2708670599,5,1.0,0.837838,37,6,31,2021-04-18,Using since 1/21/21. Today is 4/17/21. On seco...,"it’s working with time,patience and a regimen",light,brown,dry,blonde,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92373,5223641660,5,NaN,0.666667,6,2,4,2013-02-24,"Cannot believe it, but it’s really diminishing...",It really works !!!,NaN,NaN,NaN,NaN,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
92374,1299032380,5,NaN,1.000000,23,0,23,2013-02-23,Had my 2nd c-section 9 months ago and within 3...,Great for surgical scars!,NaN,NaN,NaN,NaN,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
92375,5143711471,5,NaN,0.888889,9,1,8,2013-02-21,"as stated, within the last three weeks two 2“-...",it's working!,NaN,NaN,normal,NaN,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
92376,5442226740,5,NaN,0.833333,18,3,15,2013-02-07,I have a few scars from acne and this product ...,Love it,NaN,NaN,NaN,NaN,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0


In [ ]:
summarize(check,['product_id'], numerical_cols)


,product_id_,rating_Min,rating_Max,rating_Mean,rating_Sum,is_recommended_Min,is_recommended_Max,is_recommended_Mean,is_recommended_Sum,helpfulness_Min,helpfulness_Max,helpfulness_Mean,helpfulness_Sum,total_feedback_count_Min,total_feedback_count_Max,total_feedback_count_Mean,total_feedback_count_Sum,total_neg_feedback_count_Min,total_neg_feedback_count_Max,total_neg_feedback_count_Mean,total_neg_feedback_count_Sum,total_pos_feedback_count_Min,total_pos_feedback_count_Max,total_pos_feedback_count_Mean,total_pos_feedback_count_Sum,price_usd_Min,price_usd_Max,price_usd_Mean,price_usd_Sum
0,P377368,1,5,4.05314,1678,0.0,1.0,0.7,84.0,0.0,1.0,0.642623,222.990276,0,253,10.321256,4273,0,79,1.934783,801,0,235,8.386473,3472,56.0,56.0,56.0,23184.0


In [ ]:
c2 = reviews_df.groupby(['product_id']).agg({'author_id':'nunique','review_text':'nunique'}).reset_index()
c2[c2['product_id']=='P377368']

# There should be 355 reviews, thus 355 total feedback for this single product
# However total feedback count sum goes to 4273, column is not reliable
# We will derive our own during sentiment analysis, safe to aggregate excluding these columns and avoid duplicates

,product_id,author_id,review_text
106,P377368,354,355


In [ ]:
# Checking to see if duplicated author_id are legit

author_counts = check['author_id'].value_counts()

# Find author_ids that appear exactly twice
duplicate_authors = author_counts[author_counts == 2].index

# View only those rows
check[check['author_id'].isin(duplicate_authors)]

# Each duplicated author id correspond to a unique review, good to go

,author_id,rating,is_recommended,helpfulness,total_feedback_count,total_neg_feedback_count,total_pos_feedback_count,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd
92236,8274447,5,NaN,NaN,0,0,0,2014-05-13,This product is really helping my (new) scar f...,Scar already responding,NaN,NaN,combination,NaN,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
92247,8274447,4,NaN,0.954545,22,1,21,2014-04-11,I wrote an earlier review re: this product. Th...,Kate Somerville D-Scar,NaN,NaN,combination,NaN,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
92251,2512400640,5,NaN,NaN,0,0,0,2014-03-26,I got a sample of this and i wish i had more c...,It works,NaN,NaN,oily,NaN,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0
92274,2512400640,3,NaN,0.000000,2,2,0,2013-12-26,I got this as a free sample (glad i didnt spen...,Okay...,NaN,NaN,oily,NaN,P377368,D-Scar Scar Diminishing Serum,Kate Somerville,56.0


### Clean up duplicates

In [ ]:
reviews_df_clean = reviews_df.copy(deep=True)
print(reviews_df_clean.shape)

# Columns to groupby, they will become the key to unique level in our data
key_cols = [i for i in reviews_df.columns if i not in ['helpfulness', 'total_feedback_count',	'total_neg_feedback_count',	'total_pos_feedback_count']]

# Instead of droping feedback columns altogether, sum up to the review level
reviews_df_clean = reviews_df_clean.groupby(key_cols).agg({'helpfulness':'mean', 'total_feedback_count':'sum', 'total_pos_feedback_count':'sum','total_neg_feedback_count':'sum'}).reset_index()
print(reviews_df_clean.shape)
reviews_df_clean

(1094187, 18)
(581785, 18)


,author_id,rating,is_recommended,submission_time,review_text,review_title,skin_tone,eye_color,skin_type,hair_color,product_id,product_name,brand_name,price_usd,helpfulness,total_feedback_count,total_pos_feedback_count,total_neg_feedback_count
0,538863,1,0.0,2018-11-01,One use and into the trash this went. I woke u...,one and done,fair,blue,combination,blonde,P420652,Lip Sleeping Mask Intense Hydration with Vitam...,LANEIGE,24.0,NaN,0,0,0
1,561736,5,1.0,2018-07-28,This is my nightly hero. It is the one facial...,Awesome!,light,blue,combination,blonde,P421998,Midnight Recovery Concentrate Moisturizing Fac...,Kiehl's Since 1851,56.0,0.833333,6,5,1
2,561736,5,1.0,2018-07-28,This is my nightly hero. It is the one facial...,Awesome!,light,blue,combination,blonde,P445951,Midnight Recovery Concentrate Moisturizing Fac...,Kiehl's Since 1851,30.0,0.833333,6,5,1
3,602980,5,1.0,2018-10-03,This has become a must have addition to my mor...,Brain Must!!,lightMedium,blue,combination,blonde,P423159,Brain Dust,Moon Juice,38.0,1.000000,1,1,0
4,696309,5,1.0,2019-06-17,"Well, I was a skeptic, but after trying a samp...",replacing my other serums,lightMedium,brown,dry,brown,P444222,Luxury Beauty Serum Calming Treatment,Saint Jane Beauty,125.0,1.000000,30,30,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581780,orderGen270100,5,1.0,2019-02-28,I have dry sensitive combination skin and my f...,Can’t beat the price and you can see the resul...,fair,blue,combination,blonde,P427419,Hyaluronic Acid 2% + B5 Hydrating Serum,The Ordinary,15.7,NaN,0,0,0
581781,orderGen39837,5,1.0,2018-05-18,This is great for fine lines! I’ve never revie...,Actually working on my fine lines!,light,green,normal,blonde,P429515,C-Tango Vitamin C Eye Cream,Drunk Elephant,64.0,0.250000,8,2,6
581782,orderGen39837,5,1.0,2020-08-24,I got a facial and was told I have beautiful s...,Love this product,light,green,normal,blonde,P456418,Wild Rose Night-Brightening Sleeping Facial,KORRES,50.0,0.900000,10,9,1
581783,orderGen51156,5,1.0,2020-02-10,Nice cooling sensation once applied to the ski...,cooling and moisturizing,lightMedium,brown,combination,black,P433443,Aqua Bomb Sleeping Mask,belif,38.0,0.000000,1,0,1


In [ ]:
reviews_df_clean.nunique()

,0
author_id,317168
rating,5
is_recommended,2
submission_time,2029
review_text,509491
review_title,282012
skin_tone,14
eye_color,6
skin_type,4
hair_color,7


### Save full dataframe

In [ ]:
reviews_df_clean.to_pickle(data_dir + f'reviews_data_clean_{td_date}.pkl')